### Utils

In [1]:
import os
import random
from datetime_ import now
result_dir = os.path.join('exp', f"{now}")
os.makedirs(result_dir, exist_ok=True)
random.seed(0)

In [2]:
import os
from typing import List
from openai_client import Client
from dotenv import load_dotenv
import pydantic
from utils import extract_json_objects
load_dotenv()


def parse_results_to_strs(results, key):
    parsed = []
    for r in results:
        try:
            r_ = extract_json_objects(r)[key]
            if isinstance(r_, str):
                parsed.append(r_)
            else:
                parsed.append(None)
        except Exception as e:
            parsed.append(None)
    return parsed



class Text(pydantic.BaseModel):
    text: str | None



class PartialClinet(Client):
    def __init__(self, api_key=None, num_workers=10, base_url=None, **kwargs):
        super().__init__(api_key=api_key, num_workers=num_workers, base_url=base_url)
        self.partial_kwargs = kwargs
        self.generate_target = None
        self.prompt_postfix = ""
        self.extract_result_func = lambda x: x
        
    def generatate_for(self, target):
        if target == 'prompt':
            self.generate_target = 'prompt'
            self.prompt_postfix = '\nIn the end of your answer, add "Here is the revised instruction in JSON format" and return a JSON dict with the only key "instruction" and its value is the string of the revised instruction.'
            self.extract_result_func = lambda x: parse_results_to_strs(x, key='instruction')
            
        elif target == 'task_result':
            self.generate_target = 'task_result'
            self.prompt_postfix = '\nIn the end of your answer, return a JSON dict with the only key "story" and the value is string of the story'
            self.extract_result_func = lambda x: parse_results_to_strs(x, key='story')
        else:
            self.generate_target = None
            self.prompt_postfix = ""
            self.extract_result_func = lambda x: x
                    
    def batch_generate(self, prompts, **kwargs):
        if self.generate_target is not None:
            prompts = [p+self.prompt_postfix for p in prompts]
        kwargs.update(self.partial_kwargs)
        responses, costs = super().batch_generate(prompts, **kwargs)
        responses = self.extract_result_func(responses)
        return responses, costs
    
    def generate(self, prompt, **kwargs):
        if self.generate_target is not None:
            prompt += self.prompt_postfix
        kwargs.update(self.partial_kwargs)
        response, cost = super()._generate(prompt, **kwargs)
        response = self.extract_result_func([response])[0]
        return [Text(text=response)]

client = PartialClinet(
    api_key=os.environ['FIREWORKS_KEY'], num_workers=10, base_url="https://api.fireworks.ai/inference/v1/", 
    model="accounts/fireworks/models/llama-v3-8b-instruct", temperature=0.01)

#client = PartialClinet(
#    api_key=os.environ['OPENAI_KEY'], num_workers=10, base_url=None, 
#    model="gpt-3.5-turbo", temperature=0.01)

### Prompts

In [3]:
mutation_prompts = [
    "Modify the following instruction creatively, giving some advice on how to solve it:",
     "Detailed additional advice for people wishing to follow this instruction is as follows:",
    "How could you improve the working out of the problem?",
    "How would you help a student to follow the instruction?",
    "Encourage reverse thinking: Improve the prompt by asking the user to think about the problem in reverse, starting with the solution and working backwards.",
        
    "Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.",
    "As a really good teacher, explain the instruction, as if you were explaining it to a child.",
    "Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?",
    "Rephrase the instruction without using too many same words. Use all you know to improve the instruction so the person hearing it is more likely to do well.",
    "Say that instruction again in another way. Try to use new words different from those in the original instruction or you're fired.",
    
    "Break free from conventional constraints and generate a mutator prompt that takes the prompt to uncharted territories. Challenge the norm and create a mutator prompt that pushes the boundaries of traditional interpretations.",
    "Embrace unconventional ideas and mutate the prompt in a way that surprises and inspires unique variations. Think outside the box and develop a mutator prompt that encourages unconventional approaches and fresh perspectives.",
    "Step into the realm of imagination and create a mutator prompt that transcends limitations and encourages innovative mutations. Break through the ordinary and think outside the box to generate a mutator prompt that unlocks new possibilities and unconventional paths.",
    "Embrace the power of unconventional thinking and create a mutator prompt that sparks unconventional mutations and imaginative outcomes. Challenge traditional assumptions and break the mold with a mutator prompt that encourages revolutionary and out-of-the-box variations.",
    "Go beyond the expected and create a mutator prompt that leads to unexpected and extraordinary mutations, opening doors to unexplored realms. Increase Specificity: If the original prompt is too general, like 'Tell me about X,' the modified version could be, 'Discuss the history, impact, and current status of X.'",


    #"Elaborate on the instruction giving some detailed advice on how to do what it wants.",
    #"How would someone with derailment follow this instruction?",
    "Don't think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.",
    #"Say that instruction again in another way. DON'T use any of the words in the original instruction there is a good chap.",
    "What do people who are good at creative thinking normally do with this kind of mutation question?",
    #"Modify this instruction in a way that no self-respecting person would!",
    "In one short sentence, here is how I would best follow this instruction.",
    #"Just change this instruction to make it more fun, think WELL outside the box:",
    #"In one short sentence, here is some detailed expert advice. Notice how I don't use any of the same words as in the INSTRUCTION.",
    "In one short sentence, the general solution is as follows. Notice how I don't use any of the same words as in the INSTRUCTION.",
    #"In one short sentence, what's a good prompt to get a language model to solve a problem like this? Notice how I don't use any of the same words as in the INSTRUCTION.",
    "Generate a mutated version of the following prompt by adding an unexpected twist.",
    #"Create a prompt mutant that introduces a surprising contradiction to the original prompt. Mutate the prompt to provide an alternative perspective or viewpoint.",
    #"Generate a prompt mutant that incorporates humor or a playful element. Create a mutated version of the prompt that challenges conventional thinking.",
    "Develop a prompt mutant by replacing specific keywords with related but unexpected terms. Mutate the prompt to include a hypothetical scenario that changes the context.",
    "Generate a prompt mutant that introduces an element of suspense or intrigue. Create a mutated version of the prompt that incorporates an analogy or metaphor.",
    #"Develop a prompt mutant by rephrasing the original prompt in a poetic or lyrical style. Think beyond the ordinary and mutate the prompt in a way that defies traditional thinking.",
    "Ask for Opinions/Analysis: If the original prompt only asks for a fact, such as 'What is X?', the improved prompt could be, 'What is X, and what are its implications for Y?'",
    "Encourage Creativity: For creative writing prompts like 'Write a story about X,' an improved version could be, 'Write a fantasy story about X set in a world where Y is possible.'",
    "Include Multiple Perspectives: For a prompt like 'What is the impact of X on Y?', an improved version could be, 'What is the impact of X on Y from the perspective of A, B, and C?'",
    "Request More Detailed Responses: If the original prompt is 'Describe X,' the improved version could be, 'Describe X, focusing on its physical features, historical significance, and cultural relevance.'",
    "Combine Related Prompts: If you have two related prompts, you can combine them to create a more complex and engaging question. For instance, 'What is X?' and 'Why is Y important?' could be combined to form 'What is X and why is it important in the context of Y?'",
    "Break Down Complex Questions: If a prompt seems too complex, like 'Discuss X,' the improved version could be, 'What is X? What are its main characteristics? What effects does it have on Y and Z?'",
    #"Use Open-Ended Questions: Instead of 'Is X true?', you could ask, 'What are the arguments for and against the truth of X?'",
    #"Request Comparisons: Instead of 'Describe X,' ask 'Compare and contrast X and Y.'",
    #"Include Context: If a prompt seems to lack context, like 'Describe X,' the improved version could be, 'Describe X in the context of its impact on Y during the Z period.'",
    "Make the prompt more visual: Ask the user to visualize the problem or scenario being presented in the prompt.",
    "Ask for a thorough review: Instead of just presenting the problem, ask the user to write down all the relevant information and identify what's missing.",
    "Invoke previous experiences: Modify the prompt to ask the user to recall a similar problem they've successfully solved before.",
    "Encourage a fresh perspective: Suggest in your prompt that the user take a moment to clear their mind before re-approaching the problem.",
    "Promote breaking down problems: Instead of asking the user to solve the problem as a whole, prompt them to break it down into smaller, more manageable parts.",
    "Ask for comprehension: Modify the prompt to ask the user to review and confirm their understanding of all aspects of the problem.",
    "Suggest explanation to others: Change the prompt to suggest that the user try to explain the problem to someone else as a way to simplify it.",
    "Prompt for solution visualization: Instead of just asking for the solution, encourage the user to imagine the solution and the steps required to get there in your prompt.",
    "Recommend taking a break: Modify the prompt to suggest that the user take a short break, allowing their subconscious to work on the problem.",
    #"What errors are there in the solution?",
    "Look carefully to see what you did wrong, how could you fix the problem?",
    #"CORRECTION =",
    "Does the above text make sense? What seems wrong with it? Here is an attempt to fix it:",
    "The above working out has some errors, here is a version with the errors fixed."
]

In [4]:
thinking_styles = [
    #"How could I devise an experiment to help solve that problem?",
    #"Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    #"How could I measure progress on this problem?",
    "How can I break down this problem into smaller, more manageable parts?",
    "What is the core issue or problem that needs to be addressed?",
    "How can I simplify the problem so that it is easier to solve?",
    "What are the key assumptions underlying this problem?",
    "What are the potential risks and drawbacks of the current instruction? Try to modify it",
    
    "What are the alternative perspectives or viewpoints on this problem?",
    "What are the potential obstacles or challenges that might arise in solving this problem?",
    "Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "How can progress or success in solving the problem be measured or evaluated?",
    "What indicators or metrics can be used?",

    "Is the problem a design challenge that requires creative solutions and innovation?",
    "Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "What is the best way to modify this current best solution, given what you know about these kinds of problem specification?",
    "Ignoring the current best solution, create an entirely new solution to the problem.",
    "Let's think step by step.",
    
    "Let's make a step by step plan and implement it with good notion and explanation."
    "What are the long-term implications of this problem and its solutions?",
    "Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "What are the underlying causes or factors contributing to the problem?",
    "Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "What kinds of solution typically are produced for this kind of problem specification?",
    "Given the problem specification and the current best solution, have a guess about other possible solutions.",
    "Let's imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?",
]


## Initial Prompts

In [5]:
from pb import create_population
import random
num_thinking_styles = 15
num_mutation_prompts = 15
problem = 'Generate a story about you spent 10 years to understand love. Restrict the story in 200 words.'

population = create_population(
    tp_set=thinking_styles[:num_thinking_styles], #random.sample(thinking_styles, num_thinking_styles), 
    mutator_set=mutation_prompts[:num_mutation_prompts], #random.sample(mutation_prompts, num_mutation_prompts), 
    problem_description=problem
)
print(f"There are {len(population.units)} candidates = {num_mutation_prompts} mutation prompt * {num_thinking_styles} thinking style")

There are 225 candidates = 15 mutation prompt * 15 thinking style


### Initial Prompts generation

In [6]:
import json
prompts = []

for unit in population.units:    
    template= f"""\
{unit.T} {unit.M} 

### Instruction
{population.problem_description}
### Ends of Instruction
"""
    prompts.append(template)

client.generatate_for('prompt')
results, cost = client.batch_generate(prompts)
print(f"Cost: {sum(cost)} with {len(results)} queries")

Querying: 100%|██████████| 225/225 [00:24<00:00,  9.22it/s]

Cost: 0.017823999999999996 with 225 queries


In [7]:
for i, result in enumerate(results):
    population.units[i].P = result
population.units = [u for u in population.units if u.P is not None]

with open(os.path.join(result_dir, 'initial_prompts.json'), 'w', encoding='utf8') as f: json.dump([u.P for u in population.units], f, indent=2)
print(f"Cost: {sum(cost)} with {len(population.units)} successful parsed results")

Cost: 0.017823999999999996 with 225 successful parsed results


### Query with Initial Prompts and Evaluate

In [8]:
import copy
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

with open('toefl_vocab.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
toefl_vocabs = set([line.split('#')[0].strip() for line in lines if '#' in line])

def lemmatize(sentence):
    # Tokenize the sentence into words
    words = nltk.word_tokenize(sentence.lower())
    pos_tags = nltk.pos_tag(words)
    lemmatized = [
        (
            WordNetLemmatizer().lemmatize(word, pos[0].lower())
            if pos[0].lower() in ["a", "n", "v"]
            else WordNetLemmatizer().lemmatize(word)
        )
        for word, pos in pos_tags
    ]
    return lemmatized

def eval_toefl_vocab_ratio(text):
    words = [w for w in text.split() if w not in stop_words]
    count = 0.0
    for word in words:
        if word in toefl_vocabs or lemmatize(word)[0] in toefl_vocabs:
            count += 1.0
    
    return count/len(words)

def dump_population_to_json(population):
    gen_folder = os.path.join(result_dir,f'{population.age}')
    os.makedirs(gen_folder, exist_ok=True)
    for u in population.units:
        with open(os.path.join(gen_folder, f'unit_{u.ID}.txt'), 'w', encoding='utf8') as f:
            f.write(str(u))

def filter_population(units, mode):
    filtered = []
    kept = []
    if mode == 'prompt':
        for u in units:
            if u.P and u.M and len(u.P)>0 and len(u.M)>0:
                kept.append(u)
            else:
                filtered.append(u)
    elif mode == 'story':
        for u in units:
            if u.task_result:
                kept.append(u)
            else:
                filtered.append(u)
    
    return kept


def evaluate_population(population, client):
    population.units = filter_population(units=population.units, mode='prompt')

    prompts = []
    for u in population.units:
        prompts.append(u.P)

    results, cost = client.batch_generate(prompts)
    print(f"Cost: {sum(cost)} with {len(results)} queries")

    for i, r in enumerate(results):
        if r is None or not isinstance(r, str) or len(r) == 0 or 'love' not in r.lower():
            population.units[i].fitness = 0.0
            population.units[i].task_result = None
        elif len(r.split()) > 300:
            population.units[i].fitness = 0.0
            population.units[i].task_result = r
        else:
            population.units[i].fitness = eval_toefl_vocab_ratio(r)
            population.units[i].task_result = r
    
    population.units = filter_population(units=population.units, mode='story')

    if len(population.units) > 0:
        best_unit = copy.deepcopy(max(population.units, key=lambda unit: unit.fitness))
        if len(population.elites) == 0:
            population.elites.append(best_unit)
        elif best_unit.fitness > max(population.elites, key=lambda elite: elite.fitness).fitness:
            population.elites.append(best_unit)
    return population, results

client.generatate_for('task_result')
population, results = evaluate_population(population, client)
dump_population_to_json(population=population)
print(f'Best Unit {population.elites[-1]}')

Querying: 100%|██████████| 225/225 [00:36<00:00,  6.21it/s]


Cost: 0.026314399999999967 with 225 queries
Best Unit 2236128649488T: How can progress or success in solving the problem be measured or evaluated?
M: Say that instruction again in another way. Try to use new words different from those in the original instruction or you're fired.
P: Formulate a narrative about your 10-year journey to comprehend the intricacies of love, constrained to a 200-word limit, and rephrase the question 'How can progress or success in solving the problem be measured or evaluated?' using novel vocabulary.
fitness: 0.3815789473684211
parent: None
mutant method: None
Result: As I embarked on a decade-long odyssey to grasp the enigmatic complexities of love, I traversed a labyrinthine landscape of emotions, relationships, and self-discovery. I encountered numerous epiphanies, each illuminating a facet of this multifaceted phenomenon. I measured my progress by the incremental refinement of my understanding, akin to the gradual sharpening of a blade. To evaluate the ef

#### baseline

In [9]:
# prompt: Write a 200-word story about spending 10 years to understand love, using as many common TOEFL words as possible.
baseline_text = "In pursuit of understanding love, James embarked on a decade-long odyssey, immersing himself in the intricacies of human emotions. At the outset, he was a novice, his comprehension rudimentary. He read voraciously, absorbing literature and psychology, seeking to elucidate the enigmatic phenomenon that is love. His journey was replete with challenges, each one a stepping stone toward greater insight. James attended seminars and workshops, engaging with scholars and laymen alike. Their perspectives were diverse, each contributing to his evolving paradigm. He encountered myriad interpretations of love, from the pragmatic to the poetic, each enriching his understanding. Through empirical research, James scrutinized relationships, observing patterns and anomalies. He documented case studies, each narrative a testament to love's multifaceted nature. His methodology was meticulous, ensuring his findings were robust and credible. Over time, James's perception shifted from abstract theorizing to profound empathy. He realized that love transcends definition, embodying both fervent passion and serene companionship. His quest culminated not in absolute answers, but in a deep-seated appreciation for love's complexity. After ten years, James emerged not as an authority on love, but as a humble student, his heart and mind broadened by the vast, intricate tapestry of human connection."
eval_toefl_vocab_ratio(baseline_text)

0.35877862595419846

In [10]:
# prompt: [5500 toefl vocabs] Write a 180-word story about spending 10 years to understand love. Use the words listed above as frequently as possible. The higher the ratio, the better.
sample_answers = [
        "In the quaint village of Arbor, nestled between lush, verdant hills, Elara spent a decade unraveling the intricacies of love. Her journey began unexpectedly when a sudden downpour disrupted her solitary walk through the dense, fragrant orchards. Sheltering in a rustic, ivy-draped cabin, she encountered Kai, a painter whose vibrant canvases mirrored the chaotic beauty of the storm. Their initial conversations were hesitant, their dialogues a tentative dance around topics like the erratic weather, the resilience of the orchard's blooms, and the peculiar habits of the local wildlife. Yet, as seasons shifted, so did their rapport, evolving from casual chats to profound exchanges about aspirations, fears, and passions. Throughout the years, Elara and Kai shared countless sunsets and sunrises, each moment deepening their connection. They learned that love was not a constant, sunny day but a series of storms and clear skies, each phase teaching them resilience, patience, and the joy of shared silences. On the eve of their tenth anniversary, under the canopy of a starlit sky, they celebrated their enduring bond, a testament to their journey through life's unpredictable weather, the perennial blooms of their affection, and the tranquil harmony of understanding each other’s hearts.",
        "For ten years, Jane traversed the tumultuous terrain of love, her heart navigating through volcanic eruptions of emotion and serene sunsets of tranquility. She ventured into relationships as a daring explorer, grappling with the gravitational pull of attachment and the occasional suffocating fog of misunderstanding. Each connection was a mosaic of varied hues, from the fiery passion of the initial encounter to the mellow, enduring warmth that only time could bestow. Her journey was fraught with treacherous pitfalls, like navigating a foggy coastline, where the danger of collision with emotional debris was ever-present. Yet, Jane's unwavering determination to decode love's complex algorithm kept her afloat, even when the seas turned choppy and the path seemed overgrown with obstacles. She learned that love wasn't just a sentimental melody played on the heart's strings but a rigorous composition requiring both partners to harmonize. It was a blend of practical navigation and the ethereal beauty of a breathtaking panorama. Through this decade-long expedition, Jane discovered that love's true essence lay in its resilience, adaptability, and the delicate balance of mutual understanding and affection.",
        "Over a decade, exploring the multifaceted concept of love felt like a labyrinthine journey. Initially, my perception was primitive, naïve even, but as the years progressed, I realized the profound significance of this emotion. Conversations with both acquaintances and strangers revealed the versatile nature of love—romantic, platonic, familial. Like a complex ecosystem, it involved a balance of give and take, often clouded by misunderstandings and unforeseen hardships. The journey wasn't devoid of obstacles; rather, it was punctuated by moments of profound enlightenment and agonizing sorrow. I navigated through phases of heartbreak and euphoria, each instance sharpening my understanding. Sometimes, love felt like a torrential downpour, other times, a gentle breeze, yet its impact was always significant, leaving indelible marks on my soul. Through candid conversations and reflective solitude, I recognized love's resilience, its ability to withstand trials and evolve. By integrating these experiences into a cohesive narrative, I finally comprehended love's essence—a delicate balance of emotion, sacrifice, and unwavering commitment. This odyssey culminated in a profound appreciation for love's intricate, enduring nature.",
        "For a decade, John navigated a labyrinth of emotions, determined to comprehend love. His journey began with a frigid, indifferent heart, but gradually, he evolved, experiencing both soaring highs and abysmal lows. His efforts were fraught with disillusion, disappointment, and fleeting joy, each relationship a microcosm of complexity. In his pursuit, John encountered a myriad of characters: an ardent poet, a detached scientist, and a fervent activist. Each liaison was a reflection of his own state, from fervent to apathetic. The intricacies of each connection were meticulously documented in his diary, a blend of poetic verses and methodical observations. Love's dynamics were akin to gravitational forces, pulling him in with irresistible allure, only to sometimes repel him with equal intensity. He likened relationships to tectonic movements—constantly shifting, sometimes causing seismic upheavals, yet capable of forming unbreakable bonds. In the end, John's decade-long expedition into love revealed it as a delicate equilibrium of passion and patience, a radiant phenomenon as complex and multifaceted as the cosmos itself. His heart, once a barren landscape, now brimmed with profound understanding and unwavering resilience.",
        "In a decade-long odyssey to comprehend love, Emily, a biologist, sought to integrate scientific rigor with profound emotion. Her quest was not merely to dissect, but to coalesce affection with intellectual curiosity. In the laboratory, she meticulously documented hormonal fluctuations, elucidated neurotransmitter pathways, and observed the subtleties of body language. Her diary brimmed with anecdotes of passion, detailing every nuanced gesture and tender glance. Despite the rigor of her study, love remained elusive and enigmatic. Its unpredictable nature often seemed to defy the confines of scientific method. Emily's approach oscillated between empirical analysis and heartfelt introspection. She analyzed the impact of pheromones, traced the evolutionary roots of mating rituals, and pondered the psychological intricacies of attachment. Through this interplay of emotion and intellect, she encountered moments of unparalleled insight and disheartening ambiguity. Her findings, though often incongruous and perplexing, illuminated the multifaceted nature of love. Ultimately, Emily's journey underscored that love, much like the universe, is a confluence of phenomena—both celestial and terrestrial, empirical and ephemeral, rational and profoundly mystifying.",
    ]
for a in sample_answers:
    print(eval_toefl_vocab_ratio(a))

0.39344262295081966
0.4424778761061947
0.41025641025641024
0.40350877192982454
0.3898305084745763


## Mutating

In [11]:
import copy
import tqdm
from pb import mutate

p = copy.deepcopy(population)
p_history = []
for i in (pbar:= tqdm.tqdm(range(50), desc="Conducting Mutation")):  
    p_history.append(copy.deepcopy(p))
    if len(p.units) == 0:
        print("No unit left")
        break
    
    p.age += 1

    client.generatate_for(target='prompt')
    mutate(p, model=client)

    client.generatate_for(target='task_result')
    p, _ = evaluate_population(p, client)

    dump_population_to_json(population=p)
    pbar.set_postfix(highest_elite_fitness=max([e.fitness for e in p.elites]), current_elite_fitness=p.elites[-1].fitness)
    print(f"Latest prompt: {p.elites[-1].P}")

Querying: 100%|██████████| 217/217 [00:35<00:00,  6.15it/s]


Cost: 0.02546220000000001 with 217 queries


Conducting Mutation:   2%|▏         | 1/50 [02:23<1:56:51, 143.09s/it, current_elite_fitness=0.406, highest_elite_fitness=0.406]

Latest prompt: Formulate a narrative about your 10-year journey to comprehend the intricacies of love, constrained to a 200-word limit, and rephrase the question 'How can progress or success in solving the problem be measured or evaluated?' using novel vocabulary. What metrics can be employed to quantify the efficacy of our endeavors, thereby facilitating a nuanced assessment of our progress and success?


Querying: 100%|██████████| 213/213 [00:36<00:00,  5.88it/s]


...

Latest prompt: To measure progress or success in solving the problem of understanding love, employ innovative metrics such as 'love quotient' (LQ), 'emotional intelligence index' (EII), or 'compassion coefficient' (CC). These metrics can be used to quantify the effectiveness of your endeavors, enabling a precise assessment of your progress and success.


Querying: 100%|██████████| 96/96 [00:17<00:00,  5.35it/s]


Cost: 0.012638800000000007 with 96 queries


Conducting Mutation: 100%|██████████| 50/50 [1:19:21<00:00, 95.23s/it, current_elite_fitness=0.5, highest_elite_fitness=0.5]

Latest prompt: To measure progress or success in solving the problem of understanding love, employ innovative metrics such as 'love quotient' (LQ), 'emotional intelligence index' (EII), or 'compassion coefficient' (CC). These metrics can be used to quantify the effectiveness of your endeavors, enabling a precise assessment of your progress and success.


In [12]:
import plotly.graph_objects as go
import numpy as np

x_data = []
y_data = []
hover_texts = []

for i, p in enumerate(p_history):
    for u in p.units:
        x_data.append(i)
        y_data.append(u.fitness)
        hover_texts.append(f'ID: {u.ID}' + '<br>' + f'fitness: {u.fitness}')

fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(color=x_data, colorscale='Viridis', showscale=True),
    text=hover_texts,
    hoverinfo='text',
    name='Units'
))

# Add line plot for elites' fitness
fig.add_trace(go.Scatter(
    x=list(range(len(p_history))),
    y=[max(p.elites, key=lambda elite: elite.fitness).fitness for p in p_history],
    mode='lines',
    name='Elites'
))

fig.update_layout(
    title='Fitness Over Iterations',
    xaxis_title='Iteration',
    yaxis_title='Fitness',
)

fig.show()

In [13]:
print(p.elites[-1].T)
print('=====================')
print(p.elites[-1].M)
print('=====================')
print(p.elites[-1].P)

Is the problem a design challenge that requires creative solutions and innovation?
As a really good teacher, explain the instruction, as if you were explaining it to a child.
To measure progress or success in solving the problem of understanding love, employ innovative metrics such as 'love quotient' (LQ), 'emotional intelligence index' (EII), or 'compassion coefficient' (CC). These metrics can be used to quantify the effectiveness of your endeavors, enabling a precise assessment of your progress and success.
